In [2704]:
using LinearAlgebra

In [2705]:
# Example usage:
A = randn(5, 8)

5×8 Matrix{Float64}:
 -0.799057   0.871417   0.337891   …  -0.227154  -1.28868  -0.373258
 -0.925415  -0.463447   0.0584032     -0.226391  -1.21152  -0.699729
 -0.83296   -0.213774   0.183816       0.808503  -1.66493  -0.638911
  0.481751   1.39566    0.798006      -0.207249   1.19068   1.51806
 -0.462913  -0.799561  -0.0387124      1.20794    1.3386   -0.115368

In [2706]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Input:
    U: Left Householder matrix (orthogonal)
    H: householder vectors v
    cidx: index of column where householder transform is applied
    m: number of columns of original matrix
    
    Returns (inplace) 
    U: Left Householder matrix (orthogonal)
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    #U[cidx:end, cidx:end] = U[cidx:end, cidx:end] - 2 * v * (v' * U[cidx:end, cidx:end])
    #show(stdout, "text/plain", I - 2 * v * v')
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Input:
    V: Right Householder matrix (orthogonal)
    H: householder vectors v
    ridx: index of row where householder transform is applied
    n: number of rows of original matrix
    
    Returns (inplace) 
    V: Right Householder matrix (orthogonal) 
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    
    #V[ridx:end, ridx+1:end] = V[ridx:end, ridx+1:end] - (V[ridx:end, ridx+1:n] * v) * (2 * v')
    #show(stdout, "text/plain", I - 2 * v * v')

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Input:
    A: Rectangular m x n matrix
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (A = U * B * V')
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U', B, V
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [2707]:
function givens_rotation(v)
    """ Computers the 2x2 Givens Rotation matrix for a given 2d vector v.
    
    Input:
    v: 2d vector
    
    Returns
    G:  givens rotation matrix
    """
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end


function svd_golub_reinsh!(B, U, V, maxiter, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    and
    https://link.springer.com/article/10.1007/s11075-022-01459-9
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing right Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            V[:, p:p+1] = V[:, p:p+1] * G'
            
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :]
            U[:, p:p+1] = U[:, p:p+1] * G'
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    
    if (n > m)
        # To be done "fat" matrices
    end
        
    m, n = size(B)
    
    #U = zeros(m, m) + I
    #V = zeros(n, n) + I
    maxiter_ = maxiter * log2(n * m) 
    
    for iters = 1:min(maxiter, maxiter_)
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            #print(abs(B[j-1, j]), " ", abs(B[j-1, j]) < eps, "\n")
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end
        #print("After scan k = ", k, "\n")
        
        
        #show(stdout, "text/plain", B)
        #print("\n---------------\n")
        
        if k == 1
           # Diagonal matrix, convergence obtained
           return U, S, V
        end
        
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check if where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = B[j-1, j] == 0 ? j-1 : l 
            end
            
            l = (l == k) ? 1 : l
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
end


function svd_num(A, maxiter = 80)
   """Performs singular value decompositon of a rectangular m x n matrix A.
    
    Returns 
        U: Left singular vector matrix
        S: Singular values matrix
        Vt: Right singular vector matrix
    """ 
    # Flag for transpose (case n > m)
    m, n = size(A)
    transpose_flag = false
    
    if n > m
        transpose_flag = true
    end
    
    # First main step: Find bidiagonalized matrix B
    if transpose_flag
        P_l, B, P_r = bidiagonalize(A', true);
    else
        P_l, B, P_r = bidiagonalize(A, true);
    end
    
    # Second main step: perform svd_golub_reinsh algorithm iterations
    svd_golub_reinsh!(B, P_l, P_r, maxiter)
    
    # Check if transpose
    if transpose_flag
        return P_r, B', P_l
    else
        return P_l, B, P_r
    end
    
    
    
end

svd_num (generic function with 2 methods)

In [2708]:
A

5×8 Matrix{Float64}:
 -0.799057   0.871417   0.337891   …  -0.227154  -1.28868  -0.373258
 -0.925415  -0.463447   0.0584032     -0.226391  -1.21152  -0.699729
 -0.83296   -0.213774   0.183816       0.808503  -1.66493  -0.638911
  0.481751   1.39566    0.798006      -0.207249   1.19068   1.51806
 -0.462913  -0.799561  -0.0387124      1.20794    1.3386   -0.115368

In [2709]:
P_l, B, P_r = bidiagonalize(A, true);

In [2710]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1))

3.4416913763379853e-15

In [2711]:
U, S, V = svd_num(A);

In [2712]:
show(stdout, "text/plain", U)

5×5 Matrix{Float64}:
  0.250223    0.0625722  -0.48947     0.825103    -0.114446
  0.510684    0.0967132   0.074562    7.17039e-5   0.851052
  0.7817     -0.0103598   0.361922   -0.0908704   -0.499592
  0.0328094  -0.988586    0.0495128   0.106641     0.0883079
 -0.253888    0.0965532   0.788302    0.547335     0.0722655

In [2713]:
show(stdout, "text/plain", S)

5×8 adjoint(::Matrix{Float64}) with eltype Float64:
  4.50246       5.72046e-24   3.15991e-16   1.49764e-16  -1.60896e-16  -1.20349e-17   1.5057e-16    8.68336e-17
 -7.02487e-19  -3.43253       9.22358e-19   7.23436e-17  -2.77643e-17   2.02738e-16   1.32277e-16  -2.0735e-16
  1.02758e-20   4.61769e-18  -2.14601      -4.51778e-18  -9.12889e-18   1.1715e-17   -1.01086e-16  -5.50885e-17
 -5.39803e-17  -5.27227e-19  -2.78037e-17  -1.1934        3.94651e-19   4.72833e-17   1.47589e-16   2.98389e-17
 -2.06428e-17  -1.58677e-16   4.40806e-17  -1.41615e-15   0.747062     -1.09723e-16  -4.22652e-17   4.21729e-17

In [2714]:
show(stdout, "text/plain", V')

8×8 Matrix{Float64}:
 -0.264373    0.0140048   0.0653138   0.730606    0.185118     0.0324426  -0.564898   -0.193467
  0.189894    0.420976    0.223668    0.527279   -0.328357    -0.080707    0.357871    0.465044
  0.149308    0.512417    0.0398467  -0.294557   -0.00777222  -0.619235   -0.49023     0.0542836
  0.658352   -0.376748   -0.273176    0.246786    0.413179    -0.316859    0.0439484   0.126719
 -0.362617   -0.430863   -0.0175706   0.125229   -0.393493    -0.671436    0.200899   -0.144397
  0.0633311   0.113091   -0.840181    0.0814069  -0.468476     0.153819   -0.165795    0.0070602
 -0.425211    0.382573   -0.396703    0.0631939   0.547628    -0.183743    0.421058   -0.0227845
  0.349316    0.270701    0.0768535   0.119256   -0.113721    -0.0182692   0.255781   -0.840167

In [2700]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1))

5.671930453843194e-15

In [2715]:
# Example usage 2:
A = randn(9, 7)

9×7 Matrix{Float64}:
 -1.66191     0.119437  -0.976921   …   0.384683    0.330694   2.48853
  0.55548    -1.17255   -0.0778367      0.0175039  -0.721277   0.639745
 -2.68028     1.56784   -0.275555       1.13218     0.466872  -0.11639
  1.29349     0.662585   0.424956      -0.0674404  -0.754847   1.45007
  0.625733   -0.289563   1.54927        1.43821    -0.226024  -0.067796
 -0.0518736   1.91342   -0.469759   …  -1.29806    -2.15009    0.462774
 -0.534282    0.36182    1.68552        0.581862   -0.427034   1.73531
  0.510931   -1.23868   -1.78906        1.12871     0.878771  -0.674252
 -1.28432     0.722108  -0.0509624     -0.825355    1.00122    0.422525

In [2718]:
U, S, V = svd_num(A);

In [2719]:
show(stdout, "text/plain", U)

9×9 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.503658   -0.0677928   0.174868   -0.0264555  -0.0456546  -0.0855678  -0.723613   -0.0785031   0.413847
  0.148576    0.151623    0.203909    0.580154   -0.134755   -0.126874   -0.31688    -0.417219   -0.517723
 -0.557526   -0.459269    0.235897   -0.163876   -0.371777   -0.021387    0.30345    -0.0905999  -0.395941
 -0.0389006   0.497276   -0.191249   -0.679542   -0.10467    -0.136996   -0.226191   -0.139818   -0.390403
  0.138834   -0.0102196  -0.0675579  -0.0583608  -0.271314   -0.601917    0.27713    -0.531292    0.421501
 -0.311882    0.503069   -0.244057    0.267689   -0.488904    0.436349    0.205562   -0.0628938   0.206862
 -0.306172    0.15352    -0.327655    0.301239    0.0199643  -0.619282    0.0430897   0.528408   -0.140027
  0.315643   -0.45513    -0.555968   -0.015389   -0.501986    0.0838839  -0.33172     0.109219   -0.0538059
 -0.319357   -0.174091   -0.595846    0.0896896   0.515321    0.118764    0.0421349  -0.463

In [2720]:
show(stdout, "text/plain", S)

9×7 Matrix{Float64}:
 -4.62541       1.60858e-19   1.75174e-17  -2.30617e-17  -6.37511e-18  -9.43707e-18  -5.03702e-17
  0.0           4.37589      -2.41254e-16   1.05083e-17   1.1409e-17   -8.89408e-19   8.31015e-17
 -4.25831e-17   3.28145e-18  -0.970758     -6.60481e-13   3.38565e-18  -3.5404e-17    3.71916e-16
  1.16987e-16   4.14948e-17   4.43157e-19  -1.24663      -1.13171e-17  -5.11079e-18  -8.37346e-16
 -1.16836e-16  -1.80272e-16  -3.94269e-17  -8.54617e-18   1.9375       -4.13534e-17   2.12723e-15
  8.50613e-17  -3.39855e-17  -1.85586e-16  -3.48817e-17  -6.08922e-18  -3.41459      -2.04292e-14
  4.00319e-18  -6.78671e-17  -1.49146e-16  -1.68595e-17  -6.96416e-17   3.48821e-18   3.02921
  1.12683e-16   1.24617e-16  -1.52807e-16  -9.90819e-17  -9.60935e-18   1.01799e-16  -7.20261e-17
  5.29351e-17   1.3106e-16   -1.44553e-17  -1.07601e-16   2.27387e-17   2.18591e-16  -1.47052e-16

In [2721]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.692183    0.541268   0.0184463   0.0133979  -0.0489628  -0.048195    0.47188
  0.445079    0.201714   0.279265   -0.378046   -0.348579   -0.557766    0.328175
  0.443314    0.291162  -0.154272    0.692448   -0.25141     0.247186    0.301835
  0.32722     0.536391  -0.0413983  -0.462491    0.39086     0.485701   -0.0303662
 -0.109058   -0.183462   0.427279   -0.208718   -0.603697    0.601982    0.0385334
  0.0662529  -0.228795   0.672497    0.21059     0.540586    0.0551821   0.389082
 -0.0538844   0.455899   0.511213    0.274967   -0.058588   -0.155374   -0.651716

In [2722]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1))

1.9595436384634013e-14

In [2725]:
# Example usage 3:
A = randn(7, 7)
A = A + A'

7×7 Matrix{Float64}:
  0.658439  -2.24268    -0.564005  …  -1.33409   -0.132619    -1.31127
 -2.24268   -1.02499     2.68812      -2.31489    0.117796    -0.0905978
 -0.564005   2.68812     1.7936        1.00687   -0.682744    -1.0589
 -1.57148    0.582285    0.41168       0.503409   0.642983     0.749697
 -1.33409   -2.31489     1.00687       2.96522    0.864002     1.32138
 -0.132619   0.117796   -0.682744  …   0.864002  -0.00156682  -0.5766
 -1.31127   -0.0905978  -1.0589        1.32138   -0.5766       2.00135

In [2726]:
U, S, V = svd_num(A);

In [2727]:
show(stdout, "text/plain", U)

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.474846   -0.365612    0.247037   -0.179565    0.491969    -0.54278    -0.104569
  0.0720776  -0.750057   -0.563462    0.123312   -0.26237     -0.117848    0.129607
  0.196582    0.37202    -0.574533   -0.530053    0.230614    -0.339761    0.208067
  0.455297   -0.0370073  -0.199923    0.342474    0.518796     0.0089244  -0.604027
  0.58629    -0.370736    0.42019    -0.573578   -0.00305883   0.115147   -0.000570672
  0.0895598   0.14788     0.0809033  -0.0549842  -0.602189    -0.567601   -0.525109
  0.414283    0.0682731   0.261383    0.471478    0.0644609   -0.490464    0.537016

In [2728]:
show(stdout, "text/plain", S)

7×7 Matrix{Float64}:
 -5.55536       3.36253e-13  -9.464e-20    -2.68576e-17   4.415e-17     5.40048e-16   2.19336e-16
  1.93849e-20  -4.5819       -1.99873e-15   5.25213e-18   2.5045e-17    2.26165e-16  -1.51777e-17
 -5.98956e-17  -4.65446e-19  -4.65719      -5.17946e-18   2.14514e-17   4.4621e-18    4.68425e-17
 -5.69326e-17  -8.17295e-17  -7.48679e-17  -2.67179      -2.66287e-16  -1.99986e-18   9.97179e-17
 -3.6653e-16   -3.32306e-16  -1.95509e-16   1.28644e-19   0.0682649     1.75793e-15  -1.9251e-17
  1.3798e-16    2.78306e-17   2.34907e-16   7.90485e-17  -1.17888e-18   1.19624      -1.90151e-17
 -9.53643e-17  -8.02064e-17  -1.46287e-16   6.45419e-18  -7.30391e-17   3.18917e-18   1.54371

In [2731]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.474846  -0.0720776  -0.196582  -0.455297   -0.58629      -0.0895598  -0.414283
 -0.365612  -0.750057    0.37202   -0.0370073  -0.370736      0.14788     0.0682731
 -0.247037   0.563462    0.574533   0.199923   -0.42019      -0.0809033  -0.261383
  0.179565  -0.123312    0.530053  -0.342474    0.573578      0.0549842  -0.471478
 -0.491969   0.26237    -0.230614  -0.518796    0.00305883    0.602189   -0.0644609
  0.54278    0.117848    0.339761  -0.0089244  -0.115147      0.567601    0.490464
 -0.104569   0.129607    0.208067  -0.604027   -0.000570672  -0.525109    0.537016